# Chatbot LLM minimale

1. Esporta `LLM_PROVIDER` (`openai`, `anthropic`, `gemini`), `LLM_API_KEY` e facoltativamente `LLM_MODEL`/`LLM_BASE_URL`.
2. Installa `requests` (es. `uv add requests`).
3. Avvia la cella Python seguente per chattare dal terminale del notebook con temperatura e prompt regolabili.

In [ ]:
import os
import requests

def call_llm(prompt: str, system_prompt: str = "", temperature: float = 0.7) -> str:
    provider = os.getenv("LLM_PROVIDER", "openai").lower()
    api_key = os.environ["LLM_API_KEY"]
    params = None
    if provider == "anthropic":
        url = "https://api.anthropic.com/v1/messages"
        headers = {
            "x-api-key": api_key,
            "anthropic-version": "2023-06-01",
            "content-type": "application/json",
        }
        payload = {
            "model": os.getenv("LLM_MODEL", "claude-3-haiku-20240307"),
            "max_tokens": 1024,
            "temperature": temperature,
            "messages": [{
                "role": "user",
                "content": [{"type": "text", "text": prompt}]
            }],
        }
        if system_prompt:
            payload["system"] = system_prompt
    elif provider == "gemini":
        url = "https://generativelanguage.googleapis.com/v1beta/models/{}:generateContent".format(
            os.getenv("LLM_MODEL", "gemini-1.5-flash")
        )
        headers = {"content-type": "application/json"}
        params = {"key": api_key}
        payload = {
            "contents": [{"role": "user", "parts": [{"text": prompt}]}],
            "generationConfig": {"temperature": temperature},
        }
        if system_prompt:
            payload["system_instruction"] = {"parts": [{"text": system_prompt}]}
    else:
        url = os.getenv("LLM_BASE_URL", "https://api.openai.com/v1/chat/completions")
        headers = {
            "authorization": f"Bearer {api_key}",
            "content-type": "application/json",
        }
        payload = {
            "model": os.getenv("LLM_MODEL", "gpt-3.5-turbo"),
            "temperature": temperature,
            "messages": [
                {"role": "system", "content": system_prompt or "Rispondi in modo utile."},
                {"role": "user", "content": prompt},
            ],
        }
    response = requests.post(url, headers=headers, params=params, json=payload, timeout=60)
    response.raise_for_status()
    data = response.json()
    if provider == "anthropic":
        return "".join(part["text"] for part in data["content"] if part.get("type") == "text").strip()
    if provider == "gemini":
        return data["candidates"][0]["content"]["parts"][0]["text"].strip()
    return data["choices"][0]["message"]["content"].strip()

def chat_cli() -> None:
    system_prompt = input("System prompt (lascia vuoto per nessuno): ").strip()
    temp_raw = input("Temperatura [default 0.7]: ").strip()
    temperature = float(temp_raw) if temp_raw else 0.7
    print("Scrivi 'exit' o 'quit' per uscire.")
    while True:
        user_message = input("Tu: ").strip()
        if user_message.lower() in {"exit", "quit"}:
            break
        try:
            reply = call_llm(user_message, system_prompt, temperature)
        except Exception as exc:
            print(f"Errore: {exc}")
            continue
        print(f"Bot: {reply}\n")

if __name__ == "__main__":
    chat_cli()